## Requirements

In [3]:
# Keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
# Util
import numpy as np
import matplotlib.pyplot as plt
# Excell
import openpyxl as xl

## Processing data

### Input leyer size

In [4]:
# loading an existing workbook
wb = xl.load_workbook('Geology Input for Petar Updated April 11th.xlsx')
# Select the worksheet
items_sheet = wb['Sheet1']
# make an values only iterator over the items from row 5 to 15 and col 2 to 12.
# turn it into a tuple. Take only the first element, the second one is empty
terain_properties = list(items_sheet.iter_cols(min_row=1, max_col=1, min_col=1, values_only=True))[0]
terain_properties = list(map(lambda x: x.strip().lower() , terain_properties))

num_of_properties = len(terain_properties)

### Getting training and testing data

In [5]:
data_sheet = wb['Sheet2']

river_channel_cols = tuple(data_sheet.iter_cols(min_row=4, max_row=7, min_col=3, max_col=8, values_only=True))
river_channel_cols = [tuple(xi for xi in x if xi is not None) for x in river_channel_cols]

flood_plane_cols = tuple(data_sheet.iter_cols(min_row=9, max_row=12, min_col=3, max_col=8, values_only=True))
flood_plane_cols = [tuple(xi for xi in x if xi is not None) for x in flood_plane_cols]

river_delta_cols = tuple(data_sheet.iter_cols(min_row=14, max_row=17, min_col=3, max_col=8, values_only=True))
river_delta_cols = [tuple(xi for xi in x if xi is not None) for x in river_delta_cols]

alluvial_fan_cols = tuple(data_sheet.iter_cols(min_row=19, max_row=23, min_col=3, max_col=8, values_only=True))
alluvial_fan_cols = [tuple(xi for xi in x if xi is not None) for x in alluvial_fan_cols]

desert_dune_cols = tuple(data_sheet.iter_cols(min_row=25, max_row=28, min_col=3, max_col=8, values_only=True))
desert_dune_cols = [tuple(xi for xi in x if xi is not None) for x in desert_dune_cols]

beach_cols = tuple(data_sheet.iter_cols(min_row=30, max_row=33, min_col=3, max_col=8, values_only=True))
beach_cols = [tuple(xi for xi in x if xi is not None) for x in beach_cols]

lagoon_cols = tuple(data_sheet.iter_cols(min_row=35, max_row=39, min_col=3, max_col=8, values_only=True))
lagoon_cols = [tuple(xi for xi in x if xi is not None) for x in lagoon_cols]

sh_marine_cols = tuple(data_sheet.iter_cols(min_row=41, max_row=46, min_col=3, max_col=8, values_only=True))
sh_marine_cols = [tuple(xi for xi in x if xi is not None) for x in sh_marine_cols]

reef_cols = tuple(data_sheet.iter_cols(min_row=48, max_row=52, min_col=3, max_col=8, values_only=True))
reef_cols = [tuple(xi for xi in x if xi is not None) for x in reef_cols]

deep_marine_cols = tuple(data_sheet.iter_cols(min_row=54, max_row=59, min_col=3, max_col=8, values_only=True))
deep_marine_cols = [tuple(xi for xi in x if xi is not None) for x in deep_marine_cols]

In [6]:
def prepareStrings(items):
    acc = []
    for item_list in items:
        if len(acc) == 0:
            acc = list(map(lambda x: [x.strip().lower()] ,item_list))
            continue
        
        current_acc = []
        for item in item_list:
            for line in acc:
                current_acc.append(line + [item.strip().lower()])
        acc = current_acc
    return np.asarray(acc)


all_strict_cases = (
    river_channel_cols, 
    flood_plane_cols,
    river_delta_cols, 
    alluvial_fan_cols,
    desert_dune_cols, 
    beach_cols, 
    lagoon_cols,
    sh_marine_cols,
    reef_cols,
    deep_marine_cols
)

all_strict_cases = tuple(map( lambda x: prepareStrings(x), all_strict_cases))

for clas in all_strict_cases:
    print(clas.shape)


(288, 6)
(36, 6)
(864, 6)
(640, 6)
(72, 6)
(1024, 6)
(360, 6)
(4500, 6)
(960, 6)
(3600, 6)


In [7]:
train_strings = np.concatenate(all_strict_cases)
X_train = np.zeros((train_strings.shape[0], num_of_properties),dtype='float32')
for index, st in enumerate(train_strings):
    for prop in st:
        X_train[index][terain_properties.index(prop)] = 1.0

y_train = []
for inx, data in enumerate(all_strict_cases):
    y_train = np.concatenate((y_train, np.ones(data.shape[0])*inx))

num_of_categories = len(all_strict_cases)

y_train = to_categorical(y_train, num_of_categories)

## Model

In [8]:
model = Sequential()

# model.add(Dense(252, activation="sigmoid", input_shape=(num_of_properties,)))
# model.add(Dense(252,activation="sigmoid"))
# model.add(Dense(252,activation="sigmoid"))
model.add(Dense(num_of_categories, activation='sigmoid', input_shape=(num_of_properties,)))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

history = model.fit(X_train, y_train, epochs=30, validation_data=(X_train, y_train))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1060      
Total params: 1,060
Trainable params: 1,060
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
386/386 [==============================] - 4s 6ms/step - loss: 1.3224 - accuracy: 0.7143 - val_loss: 0.7290 - val_accuracy: 0.8974
Epoch 2/30
386/386 [==============================] - 2s 5ms/step - loss: 0.5067 - accuracy: 0.9326 - val_loss: 0.3530 - val_accuracy: 0.9601
Epoch 3/30
386/386 [==============================] - 2s 5ms/step - loss: 0.2750 - accuracy: 0.9735 - val_loss: 0.2142 - val_accuracy: 0.9832
Epoch 4/30
386/386 [==============================] - 2s 5ms/step - loss: 0.1775 - accuracy: 0.9843 - val_loss: 0.1468 - val_accuracy: 0.9862
Epoch 5/30
386/386 [==============================] - 2s 6ms/step - loss: 0

In [9]:
model.save('./models/newest.model')

INFO:tensorflow:Assets written to: ./models/newest.model\assets
